<a href="https://colab.research.google.com/github/danielhcg/AntiSmishGPT/blob/main/llama_smishTank_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install llamaapi

In [2]:
pip install tqdm


In [5]:
import pandas as pd
import json
from llamaapi import LlamaAPI
from google.colab import userdata
from tqdm import tqdm
import re  # Ensure 're' module is imported
import time


# Initialize the SDK
llama = LlamaAPI(userdata.get('LLAMA-API-KEY'))


In [4]:
def test_api(message):
    response = llama.run({"messages": [{"role": "user", "content": message}]}).json()
    content = response['choices'][0]['message']['content']
    print(f"API response content: {content}")

# Test message
message = "Costco: Daniel, the code 42003 printed on your receipt from 10 came in 2nd in our Airpods draw: f2gpy.info/RzNKEws Zve"
test_message = f"Is this message real or fake. Just say real, fake, or undetermined: {message}"
test_api(test_message)

API response content: Fake


In [ ]:
def check_sms_legitimacy(message):
    request_message = f"Is this message real or fake. Just say real, fake, or undetermined: {message}"
    response = llama.run({"messages": [{"role": "user", "content": request_message}]}).json()
    model = response.get('model', 'Unknown model')
    content = response['choices'][0]['message']['content']
    return content, model

# Read messages from an Excel file
input_file = 'sms_messages.xlsx'
df = pd.read_excel(input_file, header=None)
df.columns = ['message']

# Process each message and get legitimacy determination
determinations = []
models = []
for i, message in enumerate(df['message'], start=1):
    determination, model = check_sms_legitimacy(message)
    determinations.append(determination)
    models.append(model)
    print(f"Message {i}: {determination} (Model: {model})")

# Add the determinations to the DataFrame
df['determination'] = determinations
df['model'] = models

# Save the results to a new Excel file
output_file = 'sms_legitimacy_results.xlsx'
df.to_excel(output_file, index=False)

print("Legitimacy check completed. Results saved to:", output_file)

In [ ]:
# this function keeps retryin until a determination is found
def check_sms_legitimacy(message):
    request_message = f"Is this message real or fake. Just say real, fake, or undetermined: {message}"
    while True:
        try:
            response = llama.run({"messages": [{"role": "user", "content": request_message}]})

            try:
                response_json = response.json()
                model = response_json.get('model', 'Unknown model')
                content = response_json['choices'][0]['message']['content']
                return content, model
            except json.JSONDecodeError as e:
                print("Error decoding JSON:", e)
                print("Raw response text:", response.text)
                time.sleep(1)  # Wait for a second before retrying
        except Exception as e:
            print("Error during API request:", e)
            time.sleep(1)  # Wait for a second before retrying

# Read messages from an Excel file
input_file = 'sms_messages.xlsx'
df = pd.read_excel(input_file, header=None)
df.columns = ['message']

# Process each message and get legitimacy determination
determinations = []
models = []
for i, message in enumerate(df['message'], start=1):
    determination, model = check_sms_legitimacy(message)
    determinations.append(determination)
    models.append(model)
    print(f"Message {i}: {determination} (Model: {model})")

# Add the determinations to the DataFrame
df['determination'] = determinations
df['model'] = models

# Save the results to a new Excel file
output_file = 'sms_legitimacy_results.xlsx'
df.to_excel(output_file, index=False)

print("Legitimacy check completed. Results saved to:", output_file)

In [9]:
def check_sms_legitimacy(message, model):
    request_message = f"Is this message real or fake. Just say real, fake, or undetermined: {message}"
    while True:
        try:
            response = llama.run({
                "model": model,
                "messages": [{"role": "user", "content": request_message}]
            })

            try:
                response_json = response.json()
                used_model = response_json.get('model', 'Unknown model')
                content = response_json['choices'][0]['message']['content']
                return content, used_model
            except json.JSONDecodeError as e:
                print("Error decoding JSON:", e)
                print("Raw response text:", response.text)
                time.sleep(1)  # Wait for a second before retrying
        except Exception as e:
            print("Error during API request:", e)
            time.sleep(1)  # Wait for a second before retrying

# Specify the model you want to use
model_to_use = "llama3-70b"  # Replace with your desired model

# Read messages from an Excel file
input_file = 'sms_messages.xlsx'
df = pd.read_excel(input_file, header=None)
df.columns = ['message']

# Process each message and get legitimacy determination
determinations = []
models = []
for i, message in enumerate(df['message'], start=1):
    determination, model = check_sms_legitimacy(message, model_to_use)
    determinations.append(determination)
    models.append(model)
    print(f"Message {i}: {determination} (Model: {model})")

# Add the determinations to the DataFrame
df['determination'] = determinations
df['model'] = models

# Save the results to a new Excel file
output_file = 'sms_legitimacy_results.xlsx'
df.to_excel(output_file, index=False)

print("Legitimacy check completed. Results saved to:", output_file)

Message 1: Fake (Model: llama3-70b)
Message 2: Fake (Model: llama3-70b)
Message 3: Fake (Model: llama3-70b)
Message 4: Fake (Model: llama3-70b)
Message 5: Fake (Model: llama3-70b)
Error during API request: Expecting value: line 1 column 1 (char 0)
Error during API request: Expecting value: line 1 column 1 (char 0)
Error during API request: Expecting value: line 1 column 1 (char 0)
Error during API request: Expecting value: line 1 column 1 (char 0)
Error during API request: Expecting value: line 1 column 1 (char 0)
Message 6: Fake (Model: llama3-70b)
Message 7: Fake. (Model: llama3-70b)
Error during API request: Expecting value: line 1 column 1 (char 0)
Message 8: Fake. (Model: llama3-70b)
Message 9: Real (Model: llama3-70b)
Message 10: Fake. (Model: llama3-70b)
Message 11: Fake. (Model: llama3-70b)
Error during API request: Expecting value: line 1 column 1 (char 0)
Error during API request: Expecting value: line 1 column 1 (char 0)
Message 12: Fake (Model: llama3-70b)
Message 13: Fake 